# Presentasi 1

## Crawling only link

In [1]:
import time 
from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC 


# path crome driver
driver_path = 'chromedriver-win64/chromedriver.exe'
# driver = webdriver.Chrome(executable_path=driver_path)

service = Service(driver_path)
service.start()
driver = webdriver.Chrome(service=service)

url = 'https://www.olx.co.id/mobil-bekas_c198'

try:
    driver.get(url)

    def scroll_down(driver):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.END)
        time.sleep(10)

    scroll_down(driver)

    scroll_count = 0

    scraped_links = set()  # Use a set to store the links

    while True:
        scroll_down(driver)

        try:
            wait = WebDriverWait(driver, 15)  # Adjust the timeout value as needed awalnya 10
            muat_lainnya_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-aut-id="btnLoadMore"]'))
            )
            muat_lainnya_button.click()
            time.sleep(25) #awalnya 15
        except Exception as e:
            print(f"An error occurred: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        target_li_elements = soup.find_all('li', {'data-aut-category-id':'198'})
        # print(target_li_elements)

        for target_li in target_li_elements:
            target_a_tag = target_li.find('a')

            if target_a_tag:
                link = target_a_tag['href']
                complete_link = 'https://www.olx.co.id' + link 

                if complete_link not in scraped_links:
                    print(complete_link)
                    scraped_links.add(complete_link)

        scroll_count+=1

    for link in scraped_links:
        print(link)

    with open('link_mobil_bekas.txt', 'w') as file :
        for link in scraped_links:
            file.write(link + '\n')

except Exception as e:
    print(f"An error occurred: {e}")


driver.quit()

file.close()

https://www.olx.co.id/item/olx-autos-tdp-18jt-mazda-cx-3-20-touring-bensin-at-putih-2017-iid-904830055
https://www.olx.co.id/item/olx-autos-nissan-x-trail-20-bensin-at-2017-too-iid-903995166
https://www.olx.co.id/item/olx-autos-honda-brio-satya-12-e-bensin-at-2017-kfw-iid-910556173
https://www.olx.co.id/item/olx-autos-honda-mobilio-15-e-bensin-at-2016-639-iid-911414822
https://www.olx.co.id/item/olx-autos-toyota-rush-15-trd-sportivo-bensin-at-2020-kor-iid-911039165
https://www.olx.co.id/item/dp10-km26000-520i-2020-nik2019-black-like-530i-330i-autohigh-iid-861502871
https://www.olx.co.id/item/olx-autos-tdp-15jt-toyota-avanza-15-g-bensin-mt-hitam-2022-iid-906387749
https://www.olx.co.id/item/olx-autos-honda-hrv-15-e-bensin-at-2019-syy-iid-906616824
https://www.olx.co.id/item/honda-mobilio-2015-bensin-iid-908267869
https://www.olx.co.id/item/olx-autos-tdp-17jt-honda-jazz-15-rs-bensin-at-abu-2019-iid-905955032
https://www.olx.co.id/item/olx-autos-tdp-28jt-toyota-fortuner-24-4x2-vrz-trd-sol

## Scapping link we get before (langsung masuk ke halaman dalam)

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# Path to chromedriver
driver_path = 'chromedriver-win64/chromedriver.exe'
service = Service(driver_path)
service.start()


# URL to crawl
url_file_path = 'yanti.txt'

# Function to read URLs from a text file
def read_url_from_txt(file_path):
    with open(file_path, 'r') as file:
        urls = file.readlines()
        return [url.strip() for url in urls]

# Function to extract information and save it to a CSV file
def extract_and_save_info(urls, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Merk','Title', 'Year', 'Price', 'Bahan Bakar', 'KM', 'Jenis Kendaraan'])  # Add the header
        for url in urls:
            driver = webdriver.Chrome(service=service)
            try:
                driver.get(url)
                time.sleep(10)  # Adjust the sleep time if needed

                title_element = driver.find_element(By.CSS_SELECTOR, 'h1[data-aut-id="itemTitle"]')
                string = title_element.text
                title = string.split('(')[0].strip()
                merk = title.split(' ')[0]
                title_car = title.split(' ')[-1]
                year = string.split('(')[1].replace(')', '').strip()

                # print(f'Title: {title}')
                # print(f'Year: {year}')

                price_element = driver.find_element(By.CSS_SELECTOR, 'div[data-aut-id="itemPrice"]')
                string_price = price_element.text
                price = string_price.split()[-1]
                # print(f'Price: {price}')
                
                bahan_bakar_element = driver.find_element(By.CSS_SELECTOR, 'h2[data-aut-id="itemAttribute_fuel"]')
                bahan_bakar = bahan_bakar_element.text
                
                km_element = driver.find_element(By.CSS_SELECTOR, 'div[data-aut-id="itemAttribute_mileage"]')
                string_km = km_element.text
                km = string_km.replace(' KM', '')
                
                jenis_element = driver.find_element(By.CSS_SELECTOR, 'h2[data-aut-id="itemAttribute_transmission"]')
                jenis = jenis_element.text

                writer.writerow([merk, title_car, year, price, bahan_bakar, km, jenis])
            except Exception as e:
                print(f"An error occurred for URL: {url}\nError: {e}")
            finally:
                driver.quit()

# Read URLs from the file
urls = read_url_from_txt(url_file_path)

# Extract information and save it to a CSV file
extract_and_save_info(urls, 'info.csv')


# Presentasi 2 - crawling page utama

In [4]:
import time 
from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC 


# path crome driver
driver_path = 'chromedriver-win64/chromedriver.exe'
service = Service(driver_path)
service.start()
driver = webdriver.Chrome(service=service)

url = 'https://www.olx.co.id/mobil-bekas_c198'

try:
    driver.get(url)
    scroll_count = 0

    def scroll_down(driver):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.END)
        time.sleep(10)

    scroll_down(driver)

    scroll_count = 0

    scraped_links = set()  # Use a set to store the links

    while True:
        scroll_down(driver)

        try:
            wait = WebDriverWait(driver, 15)  # Adjust the timeout value as needed awalnya 10
            muat_lainnya_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-aut-id="btnLoadMore"]'))
            )
            muat_lainnya_button.click()
            time.sleep(25) #awalnya 15
        except Exception as e:
            print(f"An error occurred: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        target_li_elements = soup.find_all('li', {'data-aut-category-id':'198'})
        # print(target_li_elements)

        for target_li in target_li_elements:
            target_a_tag = target_li.find('a')
            itemPrice = target_li.find('span', {'data-aut-id': 'itemPrice'})
            itemSubTitle = target_li.find('div', {'data-aut-id': 'itemSubTitle'})
            itemTitle = target_li.find('div', {'data-aut-id': 'itemTitle'})

            if target_a_tag:
                link = target_a_tag['href']
                complete_link = 'https://www.olx.co.id' + link 

                # take text for item
                span_item_price = itemPrice.text
                price = span_item_price.split()[-1]

                # take text for year and kilometers range
                div_item_sub_title = itemSubTitle.text
                parts = div_item_sub_title.split(' - ')
                year = parts[0]
                kilometers_range = parts[1].split(' ')[0]

                # taek text for merk and car title
                div_item_title = itemTitle.text
                merk = div_item_title.split(' ')[0]
                title_car = div_item_title.split(' ')[-1]

                if (complete_link and price and year and kilometers_range and merk and title_car) not in scraped_links:
                    result = f"{complete_link},{merk},{title_car},{year},{kilometers_range},{price}"
                    print(result)
                    scraped_links.add(result)

        scroll_count+=1

    for link in scraped_links:
        print(link)

    with open('link_mobil_bekas.txt', 'w') as file :
        for link in scraped_links:
            file.write(link + '\n')

except Exception as e:
    print(f"An error occurred: {e}")


driver.quit()

file.close()

https://www.olx.co.id/item/olx-autos-toyota-agya-10-g-trd-sportivo-bensin-2013-vfu-iid-906589941,Toyota,Agya,2013,30.000-35.000,97.000.000
https://www.olx.co.id/item/olx-autos-toyota-avanza-13-g-bensin-mt-2019-silver-iid-911334753,Toyota,Avanza,2019,85.000-90.000,158.500.000
https://www.olx.co.id/item/olx-autos-mitsubishi-xpander-15-sport-bensin-at-2017-5kka-iid-893448239,Mitsubishi,Xpander,2017,100.000-105.000,208.000.000
https://www.olx.co.id/item/olx-autos-wuling-almaz-15-at-exclusive-7-seater-bensin-2019-sim-iid-906599544,Wuling,Almaz,2019,10.000-15.000,231.000.000
https://www.olx.co.id/item/alphard-g-2018-putih-mutiara-km-70rb-tgn-1-dr-baru-full-oriser-record-iid-908951396,Toyota,Alphard,2018,70.000-75.000,870.000.000
https://www.olx.co.id/item/fortuner-vrz-trd-at-2018-iid-911284338,Toyota,Fortuner,2018,65.000-70.000,429.000.000
https://www.olx.co.id/item/olx-autos-toyota-avanza-13-g-bensin-at-2013-tmt-iid-909988306,Toyota,Avanza,2013,100.000-105.000,124.000.000
https://www.olx.co